Questions to explore
#What if online simulator with streamlit
#what would this player do if they player 90 minutes
#Trends over 8 years

In [384]:
#Libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import requests, bs4, os, sys
from unidecode import unidecode


In [332]:
############################
#####SOURCE 1: FPL DATA#####
############################

#Data Source: https://github.com/vaastav/Fantasy-Premier-League
#Originally collected using FPL player API - https://fantasy.premierleague.com/api/event/{GW}/live/
#Licence: MIT
#Data description: game week data at the player level for the past 7 premier league seasons

df_fpl_22 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2022-23/gws/merged_gw.csv')
df_fpl_21 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/gws/merged_gw.csv')
df_fpl_20 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/merged_gw.csv')
df_fpl_19 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/merged_gw.csv')
df_fpl_18 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/merged_gw.csv', encoding='latin1')
df_fpl_17 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/merged_gw.csv', encoding='latin1')
df_fpl_16 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/merged_gw.csv', encoding='latin1')

#Seasons 16-18 were encoded differently
print(df_fpl_16.columns)
print(df_fpl_17.columns)
print(df_fpl_18.columns)
print(df_fpl_19.columns)
print(df_fpl_20.columns)
print(df_fpl_21.columns)
print(df_fpl_22.columns)

Index(['name', 'assists', 'attempted_passes', 'big_chances_created',
       'big_chances_missed', 'bonus', 'bps', 'clean_sheets',
       'clearances_blocks_interceptions', 'completed_passes', 'creativity',
       'dribbles', 'ea_index', 'element', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'fixture', 'fouls', 'goals_conceded',
       'goals_scored', 'ict_index', 'id', 'influence', 'key_passes',
       'kickoff_time', 'kickoff_time_formatted', 'loaned_in', 'loaned_out',
       'minutes', 'offside', 'open_play_crosses', 'opponent_team', 'own_goals',
       'penalties_conceded', 'penalties_missed', 'penalties_saved',
       'recoveries', 'red_cards', 'round', 'saves', 'selected', 'tackled',
       'tackles', 'target_missed', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'winning_goals', 'yellow_cards', 'GW'],
      dtype='object')
Index(['name', 'assists', 'attempte

In [260]:
#Checking to see if features are the same across all seasons:

df_full_FPL = pd.concat([df_fpl_22, df_fpl_21, df_fpl_20, df_fpl_19, df_fpl_18, df_fpl_17, df_fpl_16],sort=False, ignore_index=True)


missing_values = df_full_FPL.isnull().sum()

columns_with_missing_values = missing_values[missing_values > 0].index.tolist()

print("Columns that don't appear in all dataframes:\n")
print(columns_with_missing_values)


Columns that don't appear in all dataframes:

['position', 'team', 'xP', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 'starts', 'team_a_score', 'team_h_score', 'GW', 'attempted_passes', 'big_chances_created', 'big_chances_missed', 'clearances_blocks_interceptions', 'completed_passes', 'dribbles', 'ea_index', 'errors_leading_to_goal', 'errors_leading_to_goal_attempt', 'fouls', 'id', 'key_passes', 'kickoff_time_formatted', 'loaned_in', 'loaned_out', 'offside', 'open_play_crosses', 'penalties_conceded', 'recoveries', 'tackled', 'tackles', 'target_missed', 'winning_goals']


In [379]:
#Just select players that have played more than 0 minutes
df_full_FPL = df_full_FPL.loc[(df_full_FPL["minutes"] > 0)]
df_full_FPL.shape

df_EPL_names = df_full_FPL["name"].unique()
len(df_EPL_names)

2650

In [369]:
#Convert date time to date
df_full_FPL["kickoff_time"]

# Split datetime column into date and time columns
df_full_FPL[['date', 'time']] = df_full_FPL['kickoff_time'].str.split('T', expand=True)
# Drop the time column
df_full_FPL = df_full_FPL.drop('time', axis=1)
df_full_FPL = df_full_FPL.drop('kickoff_time', axis=1)

# Show the resulting DataFrame
df_full_FPL["date"]

0         2022-08-06
1         2022-08-06
2         2022-08-06
3         2022-08-06
4         2022-08-07
             ...    
157664    2016-10-22
157665    2016-10-22
157666    2016-10-23
157667    2016-10-22
157668    2016-10-22
Name: date, Length: 157669, dtype: object

In [385]:
#Cleaning names#
df_full_FPL.loc[(df_full_FPL["name"] == "Yannick_Bolasie")]
df_full_FPL['name'] = df_full_FPL['name'].str.replace('_', ' ')
df_full_FPL['name'] = df_full_FPL['name'].apply(lambda x: unidecode(x))



In [ ]:
##################
#####UNDERSTAT####
##################

#Data Source: https://understat.com/ (Webscrapping)
#Licence: NA - Data publically available and free to use.
#Data description: game week data at the player level and team level for the past 7 premier league seasons 

#Data By team

# create urls for all seasons in the EPL

base_url = 'https://understat.com/league'
league = ['EPL']
seasons = [2022, 2021, 2020, 2019, 2018, 2017, 2016]

season_data = dict()
season_data[seasons[6]] = full_stat

for i in seasons:  
  url = base_url+'/'+league[0]+'/'+season
  res = requests.get(url)
  soup = BeautifulSoup(res.content, "lxml")

  # Relevant information under <script> tags
  scripts = soup.find_all('script')

  string_with_json_obj = ''

  # Find data for teams
  for el in scripts:
      if 'teamsData' in el.text:
        string_with_json_obj = el.text.strip()
        

  # strip unnecessary characters to get only JSON data
  ind_start = string_with_json_obj.index("('")+2
  ind_end = string_with_json_obj.index("')")
  json_data = string_with_json_obj[ind_start:ind_end]
  json_data = json_data.encode('utf8').decode('unicode_escape')

  # convert JSON data into Python dictionary
  data = json.loads(json_data)

  # Get teams and their relevant ids and put them into separate dictionary
  teams = {}
  for id in data.keys():
    teams[id] = data[id]['title']

  columns = []
  # Check the sample of values per each column
  values = []
  for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break

  # Getting data for all teams
  dataframes = {}
  for id, team in teams.items():
    teams_data = []
    for row in data[id]['history']:
      teams_data.append(list(row.values()))
      
    df = pd.DataFrame(teams_data, columns=columns)
    dataframes[team] = df

  #Manipulations to make data as in the original source¶
  for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['ppda_att'] = dataframes[team]['ppda'].apply(lambda x: x['att'])
      dataframes[team]['ppda_def'] = dataframes[team]['ppda'].apply(lambda x: x['def'])
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_att'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att'])
      dataframes[team]['oppda_def'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['def'])

  frames = []
  for team, df in dataframes.items():
      df['team'] = team
      frames.append(df)
      
  full_stat = pd.concat(frames)
  full_stat = full_stat.drop(['ppda', 'ppda_allowed'], axis=1)
  full_stat.head(10)

  full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
  full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
  full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

  full_stat.reset_index(inplace=True, drop=True)
  season_data[season] = full_stat
  
df_seasons = pd.concat(season_data)




In [355]:
##################
####PLAYER IDS####
##################

# create url for all EPL in 2022/2023 season
base_url = 'https://understat.com/league'
league = 'EPL'
seasons = ["2022", "2021", "2020", "2019", "2018", "2017", "2016"]
ids = []

for i in seasons:  
    url = base_url+'/'+league+'/'+i
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Player Data listed under the <script> tags
    scripts = soup.find_all('script')
    string_with_json_obj = ''

    # Find data for playerrs
    for el in scripts:
        if 'playersData' in el.text:
            string_with_json_obj = el.text.strip()


    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]

    json_data = json_data.encode('utf8').decode('unicode_escape')

    # convert JSON data into Python dictionary
    data = json.loads(json_data)

    for item in data:
        ids.append(item['id'])

ids = set(ids) #To make them unique
ids = list(ids) #Then back to a list to make it scriptable
len(ids)

1434

In [356]:
#Get player name in a list

# create urls for each individual player
base_url = 'https://understat.com/player'
players = ids

# Iterate through all 530 players
ids=list(map(int, ids))
list_names=[]
for i in range(len(ids)):
    url = base_url+'/'+str(players[i])
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    player_name = soup.find_all('div', class_="header-wrapper")
    for name in player_name:
        list_names.append((name.text.strip()))

len(list_names) #Check length is the same as before

1434

In [357]:
##################
####PLAYER INFO###
##################

base_url = 'https://understat.com/player'
players = ids
columns = ['goals', 'shots', 'xG', 'time', 'position', 'h_team', 'a_team',
       'h_goals', 'a_goals', 'date', 'id', 'season', 'roster_id', 'xA',
       'assists', 'key_passes', 'npg', 'npxG', 'xGChain', 'xGBuildup',
       'player_id', 'player_name']

df_all = pd.DataFrame(columns=columns)
for i in range(len(ids)):
    url = base_url+'/'+str(players[i])
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    scripts = soup.find_all('script')
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'matchesData' in el.text:
          string_with_json_obj = el.text.strip()
      
    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    # convert JSON data into Python dictionary
    new_data = json.loads(json_data)
    df_name = 'df_' + str(i)
    df_name = pd.DataFrame(new_data)
    df_name['player_id'] = str(ids[i])
    df_name['player_name'] = str(list_names[i])
    df_all = pd.concat([df_all, df_name], ignore_index=True)


In [402]:
#cleaning names in df_all[df_all["player_name"]
df_all['player_name'] = df_all['player_name'].apply(lambda x: unidecode(x))
df_all[df_all["player_name"]=="Cristiano Ronaldo"]


,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,...,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup,player_id,player_name
87562,0,3,0.386538,90,FW,Aston Villa,Manchester United,3,1,2022-11-06,...,562555,0.000000,0,0,0,0.386538,0.346365,0.018913,2371,Cristiano Ronaldo
87563,0,6,0.549009,90,FW,Manchester United,West Ham,1,0,2022-10-30,...,561004,0.000000,0,0,0,0.549009,0.549009,0.000000,2371,Cristiano Ronaldo
87564,0,1,0.072028,75,FW,Manchester United,Newcastle United,0,0,2022-10-16,...,556587,0.000000,0,0,0,0.072028,0.072028,0.000000,2371,Cristiano Ronaldo
87565,1,3,0.563077,62,Sub,Everton,Manchester United,1,2,2022-10-09,...,555258,0.056892,0,1,1,0.563077,0.687255,0.158412,2371,Cristiano Ronaldo
87566,0,2,0.154007,29,Sub,Manchester United,Arsenal,3,1,2022-09-04,...,548102,0.000000,0,0,0,0.154007,0.519170,0.519170,2371,Cristiano Ronaldo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87822,1,4,0.651241,90,AML,Villarreal,Real Madrid,0,2,2014-09-27,...,37170,0.170390,0,2,1,0.651241,0.821632,0.000000,2371,Cristiano Ronaldo
87823,4,11,2.658390,90,FW,Real Madrid,Elche,5,1,2014-09-23,...,36850,0.160417,0,3,2,1.171830,1.398950,0.274232,2371,Cristiano Ronaldo
87824,3,8,0.737171,90,FWL,Deportivo La Coruna,Real Madrid,2,8,2014-09-20,...,37227,0.000000,0,0,3,0.737171,0.707728,0.052876,2371,Cristiano Ronaldo
87825,1,5,0.898565,90,AML,Real Madrid,Atletico Madrid,1,2,2014-09-13,...,36604,0.226797,0,5,0,0.155288,0.443307,0.209220,2371,Cristiano Ronaldo


In [ ]:
#I want to replace sub with actual position
# Find all unique player names in the DataFrame with sub as a position
#subs = df_all[(df_all["position"] == "Sub")]
#sub_players = subs["player_name"].unique()


In [392]:
####SET COLUMN TYPES###
df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']] = df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']].astype(int)
df_all[['xG','xA', 'npxG', 'xGChain', 'xGBuildup']] = df_all[['xG','xA', 'npxG', 'xGChain', 'xGBuildup']].astype(float)
df_all[['player_name', 'position', 'h_team', 'a_team']]= df_all[['player_name', 'position', 'h_team', 'a_team']].astype(str)
df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']] = df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']].round(1)
df_all.head()

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,...,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup,player_id,player_name
0,0,1,0.074496,17,Sub,Crystal Palace,Bournemouth,1,1,2016-08-27,...,125744,0.000000,0,0,0,0.074496,0.213970,0.139474,467,Lewis Grabban
1,0,0,0.000000,13,Sub,West Ham,Bournemouth,1,0,2016-08-21,...,125687,0.000000,0,0,0,0.000000,0.000000,0.000000,467,Lewis Grabban
2,0,0,0.000000,21,Sub,Bournemouth,Manchester United,1,3,2016-08-14,...,124906,0.089258,1,1,0,0.000000,0.089258,0.000000,467,Lewis Grabban
3,0,0,0.000000,10,Sub,Manchester United,Bournemouth,3,1,2016-05-17,...,57742,0.000000,0,0,0,0.000000,0.000000,0.000000,467,Lewis Grabban
4,0,0,0.000000,90,FW,Bournemouth,West Bromwich Albion,1,1,2016-05-07,...,63692,0.067459,0,2,0,0.000000,0.060849,0.010438,467,Lewis Grabban


In [393]:
#Filter just for EPL teams and seasons.

EPL_teams_2022 = ['Southampton', 'Bournemouth', 'Chelsea', 'Newcastle United', 'Leicester',
       "Nottingham Forest", 'Crystal Palace', 'Wolverhampton Wanderers', 'Brentford', 'Tottenham',
       'West Ham', 'Liverpool', 'Leeds', 'Fulham', 'Brighton', 'Manchester City',
       'Manchester Utd', 'Everton', 'Arsenal', 'Aston Villa', 'Norwich', 'Watford', 'Burnley',
       'West Bromwich Albion', 'Sheffield United', 'Cardiff', 'Huddersfield', 'Swansea',
       'Stoke', 'Hull', 'Middlesbrough', 'Sunderland']

# Check if values from the list exist in h_team or a_team columns
EPL_teams = df_all['h_team'].isin(EPL_teams_2022) | df_all['a_team'].isin(EPL_teams_2022)
EPL_teams.shape
# Filter the rows in the dataframe based on the mask
df_filtered = df_all[EPL_teams & (df_all['date'] >= '2016-08-13') & (df_all['time'] >0)]
df_filtered.head()

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,...,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup,player_id,player_name
0,0,1,0.074496,17,Sub,Crystal Palace,Bournemouth,1,1,2016-08-27,...,125744,0.000000,0,0,0,0.074496,0.213970,0.139474,467,Lewis Grabban
1,0,0,0.000000,13,Sub,West Ham,Bournemouth,1,0,2016-08-21,...,125687,0.000000,0,0,0,0.000000,0.000000,0.000000,467,Lewis Grabban
2,0,0,0.000000,21,Sub,Bournemouth,Manchester United,1,3,2016-08-14,...,124906,0.089258,1,1,0,0.000000,0.089258,0.000000,467,Lewis Grabban
24,0,0,0.000000,90,DC,Chelsea,Leeds,1,0,2023-03-04,...,579544,0.000000,0,0,0,0.000000,0.424390,0.424390,1376,Kalidou Koulibaly
25,0,0,0.000000,90,DC,Tottenham,Chelsea,2,0,2023-02-26,...,578335,0.000000,0,0,0,0.000000,0.033576,0.033576,1376,Kalidou Koulibaly


In [394]:
#df_filtered has 70361 rows and df_full_FPL has 70660 rows. Need to see what is missing in the first.
#Merge the two tables on name and date columns
#Add an "indicator" column to indicate which table each row belongs to.
merged_df = pd.merge(df_filtered, df_full_FPL, left_on=['player_name', 'date'], right_on=['name', 'date'], how='outer', indicator=True)

# Select rows where the left table has missing values after the merge
new_rows_df = merged_df[merged_df['_merge'] == 'right_only'].drop('_merge', axis=1)

# Show the resulting DataFrame
print(new_rows_df)

       goals  shots  xG  time position_x h_team a_team  h_goals  a_goals  \
70600    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
70601    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
70602    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
70603    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
70604    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
...      ...    ...  ..   ...        ...    ...    ...      ...      ...   
98376    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
98377    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
98378    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
98379    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   
98380    NaN    NaN NaN   NaN        NaN    NaN    NaN      NaN      NaN   

             date  ...  loaned_in  loaned_out  offside  open_play_crosses  \
70600  202

In [403]:
new_rows_df["name"]


,name,player_name
70600,Cheick Doucoure,NaN
70601,Alisson Ramses Becker,NaN
70602,Rodrigo Hernandez,NaN
70603,Cristiano Ronaldo dos Santos Aveiro,NaN
70604,Oriol Romeu Vidal,NaN
...,...,...
98376,Oriol Romeu Vidal,NaN
98377,Patrick McNair,NaN
98378,Pedro Rodriguez Ledesma,NaN
98379,Sung-yueng Ki,NaN


In [365]:
##################
####SOCCOR API#### By team
##################
seasons_num = [2022, 2021, 2020, 2019, 2018, 2017, 2016]

import pandas as pd

URL = 'https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv'
data_538 = pd.read_csv(URL)
PL = data_538[data_538['league'] == 'Barclays Premier League']
(PL[PL['season'].isin(seasons_num)]).shape


(2660, 23)